# ProcessBuilder Fluent API Demo

This notebook demonstrates the **ProcessBuilder fluent API** for building oil and gas process simulations in NeqSim.

The fluent API provides a chainable interface for creating complex processes, making code more readable and maintainable.

## 1. Installation and Setup

In [ ]:
# Uncomment to install neqsim
# !pip install neqsim

In [ ]:
import neqsim
from neqsim.thermo import fluid, printFrame
from neqsim.process import ProcessBuilder, clearProcess
print(f"NeqSim version: {neqsim.__version__}")

## 2. Define Process Input Parameters

We define the input parameters for a typical oil/gas separation and compression process.

In [ ]:
# Process input parameters
feed_rate = 10.0  # MSm3/day
feed_temperature = 55.0  # C
feed_pressure = 60.0  # bara

# Separator conditions
sep1_pressure = 50.0  # bara (1st stage)
sep2_pressure = 15.0  # bara (2nd stage) 
sep3_pressure = 2.5   # bara (3rd stage)

# Compressor conditions
comp1_outlet_pressure = 15.0  # bara
comp2_outlet_pressure = 50.0  # bara
export_pressure = 120.0  # bara

# Cooler outlet temperatures (Kelvin)
cooler_outlet_temp = 273.15 + 35.0  # 35C in Kelvin

## 3. Create Well Fluid

Define the composition of the well fluid (oil & gas mixture).

In [ ]:
# Create well fluid with typical composition
wellFluid = fluid('pr')  # Peng-Robinson equation of state

# Add components with mole fractions
wellFluid.addComponent("nitrogen", 0.5)
wellFluid.addComponent("CO2", 2.5)
wellFluid.addComponent("methane", 70.0)
wellFluid.addComponent("ethane", 8.0)
wellFluid.addComponent("propane", 5.0)
wellFluid.addComponent("i-butane", 1.5)
wellFluid.addComponent("n-butane", 2.5)
wellFluid.addComponent("i-pentane", 1.0)
wellFluid.addComponent("n-pentane", 1.0)
wellFluid.addComponent("n-hexane", 2.0)
wellFluid.addComponent("n-heptane", 3.0)
wellFluid.addComponent("n-octane", 2.0)

wellFluid.setMixingRule(2)
wellFluid.setMultiPhaseCheck(True)

# Set feed conditions
wellFluid.setTemperature(feed_temperature, "C")
wellFluid.setPressure(feed_pressure, "bara")
wellFluid.setTotalFlowRate(feed_rate, "MSm3/day")

print("Well fluid created with feed conditions:")
print(f"Temperature: {feed_temperature} C")
print(f"Pressure: {feed_pressure} bara")
print(f"Flow rate: {feed_rate} MSm3/day")

## 4. Build Complete Oil/Gas Separation Process Using Fluent API

The ProcessBuilder provides a chainable API. Each `add_*` method returns the builder, allowing method chaining.

We build a **3-stage separation process with recompression**.

In [ ]:
# Clear any existing process
clearProcess()

# Build complete oil/gas separation and recompression process
builder = ProcessBuilder("Oil Gas Separation")

# ===== INLET AND 1ST STAGE SEPARATION =====
builder.add_stream("well stream", wellFluid)
builder.add_valve("inlet choke", inlet="well stream", pressure=sep1_pressure)
builder.add_separator("1st stage separator", inlet="inlet choke", three_phase=True)

# ===== 2ND STAGE SEPARATION (Oil from 1st stage) =====
builder.add_valve("HP-MP valve", inlet="1st stage separator.oil", pressure=sep2_pressure)
builder.add_separator("2nd stage separator", inlet="HP-MP valve")

# ===== 3RD STAGE SEPARATION (Oil from 2nd stage) =====
builder.add_valve("MP-LP valve", inlet="2nd stage separator.liquid", pressure=sep3_pressure)
builder.add_separator("3rd stage separator", inlet="MP-LP valve")

# ===== STABLE OIL PRODUCT =====
builder.add_pump("oil export pump", inlet="3rd stage separator.liquid", pressure=15.0)

# ===== 1ST STAGE RECOMPRESSION (Gas from 3rd stage -> 2nd stage pressure) =====
builder.add_cooler("LP gas cooler", inlet="3rd stage separator.gas", temperature=cooler_outlet_temp)
builder.add_separator("LP scrubber", inlet="LP gas cooler")
builder.add_compressor("LP compressor", inlet="LP scrubber.gas", pressure=sep2_pressure)

# ===== MIX WITH 2ND STAGE GAS =====
builder.add_mixer("MP gas mixer", inlets=["LP compressor", "2nd stage separator.gas"])

# ===== 2ND STAGE RECOMPRESSION (Gas from mixer -> 1st stage pressure) =====
builder.add_cooler("MP gas cooler", inlet="MP gas mixer", temperature=cooler_outlet_temp)
builder.add_separator("MP scrubber", inlet="MP gas cooler")
builder.add_compressor("MP compressor", inlet="MP scrubber.gas", pressure=sep1_pressure)

# ===== MIX WITH 1ST STAGE GAS =====
builder.add_mixer("HP gas mixer", inlets=["MP compressor", "1st stage separator.gas"])

# ===== EXPORT COMPRESSION =====
builder.add_cooler("HP gas cooler", inlet="HP gas mixer", temperature=cooler_outlet_temp)
builder.add_separator("HP scrubber", inlet="HP gas cooler")
builder.add_compressor("1st stage export comp", inlet="HP scrubber.gas", pressure=80.0)
builder.add_cooler("1st stage export cooler", inlet="1st stage export comp", temperature=cooler_outlet_temp)
builder.add_compressor("2nd stage export comp", inlet="1st stage export cooler", pressure=export_pressure)
builder.add_cooler("export gas cooler", inlet="2nd stage export comp", temperature=cooler_outlet_temp)

print("Process built with", len(builder.equipment), "equipment units")

## 5. Run the Process Simulation

In [ ]:
# Run the process simulation
builder.run()
print("Process simulation completed!")

## 6. Get Results - Compressor Powers

In [ ]:
# Get compressor powers
lp_comp = builder.get("LP compressor")
mp_comp = builder.get("MP compressor")
exp_comp1 = builder.get("1st stage export comp")
exp_comp2 = builder.get("2nd stage export comp")

print("=" * 50)
print("COMPRESSOR RESULTS")
print("=" * 50)
print(f"LP Compressor Power:           {lp_comp.getPower('kW'):>10.2f} kW")
print(f"LP Compressor Outlet Temp:     {lp_comp.getOutStream().getTemperature('C'):>10.2f} C")
print()
print(f"MP Compressor Power:           {mp_comp.getPower('kW'):>10.2f} kW")
print(f"MP Compressor Outlet Temp:     {mp_comp.getOutStream().getTemperature('C'):>10.2f} C")
print()
print(f"1st Stage Export Comp Power:   {exp_comp1.getPower('kW'):>10.2f} kW")
print(f"1st Stage Export Outlet Temp:  {exp_comp1.getOutStream().getTemperature('C'):>10.2f} C")
print()
print(f"2nd Stage Export Comp Power:   {exp_comp2.getPower('kW'):>10.2f} kW")
print(f"2nd Stage Export Outlet Temp:  {exp_comp2.getOutStream().getTemperature('C'):>10.2f} C")
print()
total_power = lp_comp.getPower('kW') + mp_comp.getPower('kW') + exp_comp1.getPower('kW') + exp_comp2.getPower('kW')
print(f"TOTAL COMPRESSION POWER:       {total_power:>10.2f} kW")
print(f"                               {total_power/1000:>10.2f} MW")

## 7. Get Results - Cooler Duties

In [ ]:
# Get cooler duties (negative = cooling)
lp_cooler = builder.get("LP gas cooler")
mp_cooler = builder.get("MP gas cooler")
hp_cooler = builder.get("HP gas cooler")
exp_cooler1 = builder.get("1st stage export cooler")
exp_cooler = builder.get("export gas cooler")

print("=" * 50)
print("COOLER DUTIES")
print("=" * 50)
print(f"LP Gas Cooler:              {lp_cooler.getDuty()/1000:>10.2f} kW")
print(f"MP Gas Cooler:              {mp_cooler.getDuty()/1000:>10.2f} kW")
print(f"HP Gas Cooler:              {hp_cooler.getDuty()/1000:>10.2f} kW")
print(f"1st Stage Export Cooler:    {exp_cooler1.getDuty()/1000:>10.2f} kW")
print(f"Export Gas Cooler:          {exp_cooler.getDuty()/1000:>10.2f} kW")
print()
total_cooling = (lp_cooler.getDuty() + mp_cooler.getDuty() + hp_cooler.getDuty() + 
                 exp_cooler1.getDuty() + exp_cooler.getDuty()) / 1000
print(f"TOTAL COOLING DUTY:         {total_cooling:>10.2f} kW")

## 8. Get Results - Product Streams

In [ ]:
# Get product stream info
oil_pump = builder.get("oil export pump")
export_cooler = builder.get("export gas cooler")

print("=" * 50)
print("PRODUCT STREAMS")
print("=" * 50)
print("\nSTABLE OIL:")
print(f"  Temperature:  {oil_pump.getOutStream().getTemperature('C'):.2f} C")
print(f"  Pressure:     {oil_pump.getOutStream().getPressure('bara'):.2f} bara")
print(f"  Flow rate:    {oil_pump.getOutStream().getFlowRate('kg/hr'):.2f} kg/hr")

print("\nEXPORT GAS:")
print(f"  Temperature:  {export_cooler.getOutStream().getTemperature('C'):.2f} C")
print(f"  Pressure:     {export_cooler.getOutStream().getPressure('bara'):.2f} bara")
print(f"  Flow rate:    {export_cooler.getOutStream().getFlowRate('MSm3/day'):.4f} MSm3/day")

## 9. Print Fluid Tables

In [ ]:
# Print export gas composition
print("EXPORT GAS COMPOSITION:")
printFrame(export_cooler.getOutStream().getFluid())

In [ ]:
# Print stable oil composition
print("STABLE OIL COMPOSITION:")
printFrame(oil_pump.getOutStream().getFluid())

## 10. Parametric Study - Effect of 3rd Stage Pressure

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Study effect of 3rd stage separator pressure on total compression power
sep3_pressures = np.linspace(1.5, 5.0, 10)
total_powers = []
gas_rates = []

for p3 in sep3_pressures:
    # Create fresh fluid
    test_fluid = fluid('pr')
    test_fluid.addComponent("nitrogen", 0.5)
    test_fluid.addComponent("CO2", 2.5)
    test_fluid.addComponent("methane", 70.0)
    test_fluid.addComponent("ethane", 8.0)
    test_fluid.addComponent("propane", 5.0)
    test_fluid.addComponent("i-butane", 1.5)
    test_fluid.addComponent("n-butane", 2.5)
    test_fluid.addComponent("i-pentane", 1.0)
    test_fluid.addComponent("n-pentane", 1.0)
    test_fluid.addComponent("n-hexane", 2.0)
    test_fluid.addComponent("n-heptane", 3.0)
    test_fluid.addComponent("n-octane", 2.0)
    test_fluid.setMixingRule(2)
    test_fluid.setMultiPhaseCheck(True)
    test_fluid.setTemperature(55.0, "C")
    test_fluid.setPressure(60.0, "bara")
    test_fluid.setTotalFlowRate(10.0, "MSm3/day")
    
    # Build process with this LP pressure
    clearProcess()
    b = ProcessBuilder("Study")
    b.add_stream("well", test_fluid)
    b.add_valve("v1", inlet="well", pressure=50.0)
    b.add_separator("sep1", inlet="v1", three_phase=True)
    b.add_valve("v2", inlet="sep1.oil", pressure=15.0)
    b.add_separator("sep2", inlet="v2")
    b.add_valve("v3", inlet="sep2.liquid", pressure=p3)
    b.add_separator("sep3", inlet="v3")
    
    # Recompression
    b.add_cooler("c1", inlet="sep3.gas", temperature=273.15+35)
    b.add_separator("s1", inlet="c1")
    b.add_compressor("comp1", inlet="s1.gas", pressure=15.0)
    b.add_mixer("m1", inlets=["comp1", "sep2.gas"])
    b.add_cooler("c2", inlet="m1", temperature=273.15+35)
    b.add_separator("s2", inlet="c2")
    b.add_compressor("comp2", inlet="s2.gas", pressure=50.0)
    b.add_mixer("m2", inlets=["comp2", "sep1.gas"])
    b.add_cooler("c3", inlet="m2", temperature=273.15+35)
    b.add_separator("s3", inlet="c3")
    b.add_compressor("comp3", inlet="s3.gas", pressure=80.0)
    b.add_cooler("c4", inlet="comp3", temperature=273.15+35)
    b.add_compressor("comp4", inlet="c4", pressure=120.0)
    b.add_cooler("c5", inlet="comp4", temperature=273.15+35)
    
    b.run()
    
    power = (b.get("comp1").getPower('kW') + b.get("comp2").getPower('kW') + 
             b.get("comp3").getPower('kW') + b.get("comp4").getPower('kW'))
    gas_rate = b.get("c5").getOutStream().getFlowRate('MSm3/day')
    
    total_powers.append(power / 1000)  # MW
    gas_rates.append(gas_rate)

print(f"Completed {len(sep3_pressures)} simulations")

In [ ]:
# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Power vs pressure
ax1.plot(sep3_pressures, total_powers, 'b-o', linewidth=2, markersize=8)
ax1.set_xlabel('3rd Stage Separator Pressure (bara)', fontsize=12)
ax1.set_ylabel('Total Compression Power (MW)', fontsize=12)
ax1.set_title('Total Compression Power vs LP Separator Pressure', fontsize=14)
ax1.grid(True, alpha=0.3)

# Gas rate vs pressure
ax2.plot(sep3_pressures, gas_rates, 'g-s', linewidth=2, markersize=8)
ax2.set_xlabel('3rd Stage Separator Pressure (bara)', fontsize=12)
ax2.set_ylabel('Export Gas Rate (MSm3/day)', fontsize=12)
ax2.set_title('Export Gas Flow vs LP Separator Pressure', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Alternative: Using ProcessContext (Context Manager Pattern)

In [ ]:
from neqsim.process import ProcessContext

# Create a simple fluid
simple_fluid = fluid('srk')
simple_fluid.addComponent("methane", 90.0)
simple_fluid.addComponent("ethane", 7.0)
simple_fluid.addComponent("propane", 3.0)
simple_fluid.setMixingRule(2)
simple_fluid.setTemperature(30.0, "C")
simple_fluid.setPressure(40.0, "bara")
simple_fluid.setTotalFlowRate(5.0, "MSm3/day")

# Use context manager pattern
with ProcessContext() as ctx:
    # Create process equipment
    feed = ctx.stream("feed", simple_fluid)
    sep = ctx.separator("inlet separator", feed)
    comp = ctx.compressor("compressor", sep.getGasOutStream(), pres=100.0)
    cool = ctx.cooler("cooler", comp.getOutStream(), T=308.15)  # 35C
    
    # Run the process
    ctx.run()
    
    # Print results
    print(f"Compressor power: {comp.getPower('kW'):.2f} kW")
    print(f"Compressor outlet temp: {comp.getOutStream().getTemperature('C'):.2f} C")
    print(f"Cooler duty: {cool.getDuty()/1000:.2f} kW")

## 12. Building a Complex Process from YAML Configuration

One of the powerful features of ProcessBuilder is the ability to construct processes from configuration data. This is particularly useful for:
- Storing process configurations in version control
- Building processes dynamically from external sources
- Separating process design from code

In this example, we'll build a complete **gas compression and dehydration process** from a YAML file.

In [ ]:
import yaml

# Define a complex process configuration in YAML format
yaml_config = """
# Gas Compression and Dehydration Process Configuration
# =====================================================

process:
  name: "Offshore Gas Processing"
  description: "Multi-stage compression with TEG dehydration"

# Fluid compositions (mole fractions will be normalized)
fluids:
  wellstream:
    model: "pr"
    mixing_rule: 2
    components:
      nitrogen: 1.0
      CO2: 3.5
      methane: 78.0
      ethane: 8.5
      propane: 4.0
      i-butane: 1.2
      n-butane: 1.8
      i-pentane: 0.6
      n-pentane: 0.5
      n-hexane: 0.5
      n-heptane: 0.25
      water: 0.15
    conditions:
      temperature: 45.0  # C
      pressure: 35.0     # bara
      flow_rate: 8.0     # MSm3/day
      flow_unit: "MSm3/day"

# Equipment sequence
equipment:
  # Inlet separation
  - type: stream
    name: inlet_stream
    fluid: wellstream
    
  - type: separator
    name: inlet_separator
    inlet: inlet_stream
    description: "Inlet slug catcher / separator"
    
  # First stage compression  
  - type: gas_scrubber
    name: scrubber_1
    inlet: inlet_separator.gas
    description: "1st stage suction scrubber"
    
  - type: compressor
    name: compressor_1
    inlet: scrubber_1.gas
    outlet_pressure: 65.0
    isentropic_efficiency: 0.75
    description: "1st stage compressor"
    
  - type: cooler
    name: aftercooler_1
    inlet: compressor_1
    outlet_temperature: 40.0
    description: "1st stage aftercooler"
    
  - type: gas_scrubber
    name: ko_drum_1
    inlet: aftercooler_1
    description: "1st stage KO drum"

  # Second stage compression
  - type: compressor
    name: compressor_2
    inlet: ko_drum_1.gas
    outlet_pressure: 120.0
    isentropic_efficiency: 0.75
    description: "2nd stage compressor"
    
  - type: cooler
    name: aftercooler_2
    inlet: compressor_2
    outlet_temperature: 40.0
    description: "2nd stage aftercooler"
    
  - type: gas_scrubber
    name: ko_drum_2
    inlet: aftercooler_2
    description: "2nd stage KO drum"

  # Third stage compression (export pressure)
  - type: compressor
    name: compressor_3
    inlet: ko_drum_2.gas
    outlet_pressure: 200.0
    isentropic_efficiency: 0.75
    description: "Export compressor"
    
  - type: cooler
    name: export_cooler
    inlet: compressor_3
    outlet_temperature: 45.0
    description: "Export gas cooler"

  # Condensate handling - collect from all scrubbers
  - type: mixer
    name: condensate_mixer
    inlets:
      - inlet_separator.liquid
      - scrubber_1.liquid
      - ko_drum_1.liquid
      - ko_drum_2.liquid
    description: "Condensate collection header"
    
  - type: pump
    name: condensate_pump
    inlet: condensate_mixer
    outlet_pressure: 40.0
    description: "Condensate export pump"

# Results to extract after simulation
outputs:
  - equipment: compressor_1
    properties: [power_kW, outlet_temperature_C, compression_ratio]
  - equipment: compressor_2
    properties: [power_kW, outlet_temperature_C, compression_ratio]
  - equipment: compressor_3
    properties: [power_kW, outlet_temperature_C, compression_ratio]
  - equipment: export_cooler
    properties: [duty_kW, outlet_flow_MSm3day]
  - equipment: condensate_pump
    properties: [power_kW, outlet_flow_m3hr]
"""

# Parse the YAML
config = yaml.safe_load(yaml_config)
print("Loaded configuration for:", config['process']['name'])
print("Description:", config['process']['description'])
print(f"\nFluids defined: {list(config['fluids'].keys())}")
print(f"Equipment count: {len(config['equipment'])}")

In [ ]:
def create_fluid_from_config(fluid_config: dict):
    """Create a NeqSim fluid from YAML configuration."""
    # Create fluid with equation of state
    f = fluid(fluid_config.get('model', 'srk'))
    
    # Add components
    for component, amount in fluid_config['components'].items():
        f.addComponent(component, float(amount))
    
    # Set mixing rule
    if 'mixing_rule' in fluid_config:
        f.setMixingRule(fluid_config['mixing_rule'])
    
    # Enable multiphase if water present
    if 'water' in fluid_config['components']:
        f.setMultiPhaseCheck(True)
    
    # Set conditions
    conditions = fluid_config.get('conditions', {})
    if 'temperature' in conditions:
        f.setTemperature(conditions['temperature'], 'C')
    if 'pressure' in conditions:
        f.setPressure(conditions['pressure'], 'bara')
    if 'flow_rate' in conditions:
        flow_unit = conditions.get('flow_unit', 'kg/hr')
        f.setTotalFlowRate(conditions['flow_rate'], flow_unit)
    
    return f

def build_process_from_config(config: dict):
    """Build a process from YAML configuration dictionary."""
    from neqsim.process import ProcessBuilder, clearProcess
    
    # Clear any previous process
    clearProcess()
    
    # Create fluids
    fluids = {}
    for name, fluid_config in config.get('fluids', {}).items():
        fluids[name] = create_fluid_from_config(fluid_config)
        print(f"  Created fluid: {name}")
    
    # Create process builder
    process_name = config.get('process', {}).get('name', 'Process')
    builder = ProcessBuilder(process_name)
    
    # Add equipment from configuration
    for equip in config.get('equipment', []):
        equip_type = equip['type']
        name = equip['name']
        description = equip.get('description', '')
        
        # Handle different equipment types
        if equip_type == 'stream':
            fluid_name = equip['fluid']
            builder.add_stream(name, fluids[fluid_name])
            
        elif equip_type == 'separator':
            three_phase = equip.get('three_phase', False)
            if three_phase:
                builder.add_three_phase_separator(name, equip['inlet'])
            else:
                builder.add_separator(name, equip['inlet'])
                
        elif equip_type == 'gas_scrubber':
            builder.add_gas_scrubber(name, equip['inlet'])
            
        elif equip_type == 'compressor':
            builder.add_compressor(
                name, 
                equip['inlet'],
                outlet_pressure=equip.get('outlet_pressure'),
                isentropic_efficiency=equip.get('isentropic_efficiency', 0.75)
            )
            
        elif equip_type == 'cooler':
            builder.add_cooler(name, equip['inlet'], 
                              outlet_temperature=equip.get('outlet_temperature'))
            
        elif equip_type == 'heater':
            builder.add_heater(name, equip['inlet'],
                              outlet_temperature=equip.get('outlet_temperature'))
            
        elif equip_type == 'valve':
            builder.add_valve(name, equip['inlet'],
                             outlet_pressure=equip.get('outlet_pressure'))
            
        elif equip_type == 'pump':
            builder.add_pump(name, equip['inlet'],
                           outlet_pressure=equip.get('outlet_pressure'))
            
        elif equip_type == 'mixer':
            builder.add_mixer(name, inlets=equip.get('inlets', []))
            
        elif equip_type == 'splitter':
            builder.add_splitter(name, equip['inlet'],
                                split_fractions=equip.get('split_fractions'))
            
        elif equip_type == 'pipe':
            builder.add_pipe(name, equip['inlet'],
                           length=equip.get('length'),
                           diameter=equip.get('diameter'))
            
        elif equip_type == 'heat_exchanger':
            builder.add_heat_exchanger(name,
                                      hot_inlet=equip.get('hot_inlet'),
                                      cold_inlet=equip.get('cold_inlet'))
        else:
            # Try generic add method
            kwargs = {k: v for k, v in equip.items() 
                     if k not in ['type', 'name', 'description']}
            builder.add(equip_type, name, **kwargs)
        
        if description:
            print(f"  Added {equip_type}: {name} - {description}")
        else:
            print(f"  Added {equip_type}: {name}")
    
    return builder, fluids

# Build the process
print("Building process from YAML configuration...")
print("-" * 50)
print("Creating fluids:")
builder, fluids = build_process_from_config(config)

In [ ]:
# Run the simulation
print("\nRunning process simulation...")
builder.run()
print("Simulation complete!")

In [ ]:
def extract_equipment_results(builder, outputs_config: list):
    """Extract results from equipment based on configuration."""
    results = {}
    
    for output in outputs_config:
        equip_name = output['equipment']
        equip = builder.get(equip_name)
        
        if equip is None:
            print(f"Warning: Equipment '{equip_name}' not found")
            continue
            
        equip_results = {}
        
        for prop in output.get('properties', []):
            try:
                if prop == 'power_kW':
                    equip_results[prop] = equip.getPower('kW')
                elif prop == 'outlet_temperature_C':
                    equip_results[prop] = equip.getOutletStream().getTemperature('C')
                elif prop == 'inlet_temperature_C':
                    equip_results[prop] = equip.getInletStream().getTemperature('C')
                elif prop == 'outlet_pressure_bara':
                    equip_results[prop] = equip.getOutletStream().getPressure('bara')
                elif prop == 'inlet_pressure_bara':
                    equip_results[prop] = equip.getInletStream().getPressure('bara')
                elif prop == 'compression_ratio':
                    p_out = equip.getOutletStream().getPressure('bara')
                    p_in = equip.getInletStream().getPressure('bara')
                    equip_results[prop] = p_out / p_in
                elif prop == 'duty_kW':
                    equip_results[prop] = abs(equip.getDuty()) / 1000
                elif prop == 'outlet_flow_MSm3day':
                    equip_results[prop] = equip.getOutletStream().getFlowRate('MSm3/day')
                elif prop == 'outlet_flow_m3hr':
                    equip_results[prop] = equip.getOutletStream().getFlowRate('m3/hr')
                elif prop == 'outlet_flow_kghr':
                    equip_results[prop] = equip.getOutletStream().getFlowRate('kg/hr')
                else:
                    equip_results[prop] = f"Unknown property: {prop}"
            except Exception as e:
                equip_results[prop] = f"Error: {e}"
        
        results[equip_name] = equip_results
    
    return results

# Extract results based on YAML configuration
results = extract_equipment_results(builder, config.get('outputs', []))

# Display results
print("\n" + "=" * 60)
print("PROCESS SIMULATION RESULTS")
print("=" * 60)

for equip_name, props in results.items():
    print(f"\n{equip_name}:")
    for prop, value in props.items():
        if isinstance(value, float):
            print(f"  {prop}: {value:.2f}")
        else:
            print(f"  {prop}: {value}")

In [ ]:
# Create a summary table
import pandas as pd

# Compressor summary
compressor_data = []
for i in [1, 2, 3]:
    comp = builder.get(f'compressor_{i}')
    compressor_data.append({
        'Stage': i,
        'Inlet P (bara)': comp.getInletStream().getPressure('bara'),
        'Outlet P (bara)': comp.getOutletStream().getPressure('bara'),
        'Ratio': comp.getOutletStream().getPressure('bara') / comp.getInletStream().getPressure('bara'),
        'Inlet T (°C)': comp.getInletStream().getTemperature('C'),
        'Outlet T (°C)': comp.getOutletStream().getTemperature('C'),
        'Power (kW)': comp.getPower('kW'),
        'Efficiency': comp.getIsentropicEfficiency()
    })

comp_df = pd.DataFrame(compressor_data)
print("\nCOMPRESSOR PERFORMANCE SUMMARY")
print("-" * 80)
print(comp_df.to_string(index=False))

# Calculate totals
total_power = comp_df['Power (kW)'].sum()
print(f"\nTotal Compression Power: {total_power:.1f} kW ({total_power/1000:.2f} MW)")

In [ ]:
# Visualize the compression stages
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Pressure profile
stages = ['Inlet', 'Stage 1', 'IC 1', 'Stage 2', 'IC 2', 'Stage 3', 'Export']
pressures = [
    builder.get('inlet_stream').getOutletStream().getPressure('bara'),
    builder.get('compressor_1').getOutletStream().getPressure('bara'),
    builder.get('aftercooler_1').getOutletStream().getPressure('bara'),
    builder.get('compressor_2').getOutletStream().getPressure('bara'),
    builder.get('aftercooler_2').getOutletStream().getPressure('bara'),
    builder.get('compressor_3').getOutletStream().getPressure('bara'),
    builder.get('export_cooler').getOutletStream().getPressure('bara'),
]
axes[0].plot(stages, pressures, 'b-o', linewidth=2, markersize=10)
axes[0].set_ylabel('Pressure (bara)', fontsize=12)
axes[0].set_title('Pressure Profile', fontsize=14)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Temperature profile
temperatures = [
    builder.get('inlet_stream').getOutletStream().getTemperature('C'),
    builder.get('compressor_1').getOutletStream().getTemperature('C'),
    builder.get('aftercooler_1').getOutletStream().getTemperature('C'),
    builder.get('compressor_2').getOutletStream().getTemperature('C'),
    builder.get('aftercooler_2').getOutletStream().getTemperature('C'),
    builder.get('compressor_3').getOutletStream().getTemperature('C'),
    builder.get('export_cooler').getOutletStream().getTemperature('C'),
]
axes[1].plot(stages, temperatures, 'r-s', linewidth=2, markersize=10)
axes[1].set_ylabel('Temperature (°C)', fontsize=12)
axes[1].set_title('Temperature Profile', fontsize=14)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Power distribution
comp_names = ['Stage 1', 'Stage 2', 'Stage 3']
powers = [builder.get(f'compressor_{i}').getPower('kW') for i in [1, 2, 3]]
colors = ['#2ecc71', '#3498db', '#9b59b6']
bars = axes[2].bar(comp_names, powers, color=colors, edgecolor='black')
axes[2].set_ylabel('Power (kW)', fontsize=12)
axes[2].set_title('Compressor Power Distribution', fontsize=14)
axes[2].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, power in zip(bars, powers):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                f'{power:.0f}', ha='center', fontsize=11)

plt.tight_layout()
plt.show()

### Saving Configuration to a YAML File

In practice, you would save your process configuration to a file for reuse:

In [ ]:
# Save the configuration to a file
config_filename = 'gas_compression_process.yaml'

with open(config_filename, 'w') as f:
    f.write(yaml_config)

print(f"Configuration saved to: {config_filename}")
print(f"\nTo load and run this process later:")
print(f"""
    import yaml
    from neqsim.process import ProcessBuilder
    
    with open('{config_filename}', 'r') as f:
        config = yaml.safe_load(f)
    
    builder, fluids = build_process_from_config(config)
    builder.run()
""")

### Benefits of YAML Configuration Approach

| Benefit | Description |
|---------|-------------|
| **Version Control** | Track process changes in Git alongside code |
| **Separation of Concerns** | Process design separate from simulation logic |
| **Reusability** | Same config can be used by different scripts |
| **Validation** | Easy to add JSON Schema validation for configs |
| **Documentation** | YAML comments serve as inline documentation |
| **Parametric Studies** | Easy to modify parameters programmatically |
| **Collaboration** | Process engineers can edit YAML without Python knowledge |

## Summary

The **ProcessBuilder fluent API** provides:

1. **Chainable methods** - Build complex processes with readable chains
2. **Named equipment** - Access any equipment by name using `builder.get("name")`
3. **Dot notation for outlets** - Use `'separator.gas'`, `'separator.oil'`, `'separator.liquid'` to connect specific outlets
4. **Three-phase separators** - Use `three_phase=True` in `add_separator()` or `add_three_phase_separator()`
5. **Easy parametric studies** - Clean syntax for sensitivity analysis

### Available Methods

| Method | Description |
|--------|-------------|
| `add_stream(name, fluid)` | Add inlet stream |
| `add_separator(name, inlet, three_phase=False)` | Two or three-phase separator |
| `add_valve(name, inlet, pressure)` | Throttling valve |
| `add_compressor(name, inlet, pressure, efficiency=0.75)` | Gas compressor |
| `add_cooler(name, inlet, temperature)` | Cooler (temp in Kelvin) |
| `add_heater(name, inlet, temperature)` | Heater (temp in Kelvin) |
| `add_pump(name, inlet, pressure)` | Liquid pump |
| `add_mixer(name, inlets=[...])` | Stream mixer |
| `add_splitter(name, inlet, split_factors=[...])` | Stream splitter |
| `add_gas_scrubber(name, inlet)` | Gas scrubber |
| `run()` | Run the process simulation |
| `get(name)` | Get equipment by name |

### Dot Notation for Outlets

| Notation | Outlet Stream |
|----------|---------------|
| `'separator.gas'` or `'separator.vapor'` | Gas outlet |
| `'separator.liquid'` | Liquid outlet (2-phase) |
| `'separator.oil'` | Oil outlet (3-phase) |
| `'separator.water'` | Water outlet (3-phase) |